In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
from pathlib import Path
import twitter
from ftplib import FTP
from shutil import move

import xarray as xr
import pandas as pd
import numpy as np
from argopy import DataFetcher as ArgoDataFetcher
from argopy.stores import indexstore, indexfilter_box
import argopy
from argopy.utilities import load_dict, mapp_dict

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
# from matplotlib import image
from PIL import Image, ImageFont, ImageDraw
import textwrap

import cmocean
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
land_feature = cfeature.NaturalEarthFeature(category='physical', name='land',
                                                scale='50m', facecolor=[0.4, 0.6, 0.7])
import seaborn as sns
sns.set_style("whitegrid")
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import warnings; warnings.simplefilter('ignore')

In [2]:
def download_one_profile_data(start, end):
    """Load a random Argo profile reported over the last 24 hours"""

    # ## Download this week index
    # ftp://ftp.ifremer.fr/ifremer/argo/ar_index_this_week_prof.txt
    index_file = './ar_index_this_week_prof.txt'

    with FTP("ftp.ifremer.fr") as ftp:
        ftp.login('anonymous','anonymous')
        ftp.cwd('/ifremer/argo/')
        gFile = open(index_file, "wb")
        ftp.retrbinary('RETR ar_index_this_week_prof.txt', gFile.write)
        gFile.close()

    # ## Randomly select one profile

    filt = indexfilter_box(BOX=[-180, 180, -90, 90, start, end])
    # print(filt.BOX)
    this_df = indexstore(index_file=index_file).read_csv(filt)
    this_df['wmo'] = this_df['file'].apply(lambda x: int(x.split('/')[1]))
    print("%i Argo profiles reported between %s and %s" % (this_df.shape[0], start, end))

    # Gather coordinate of all profiles
    week_lon = this_df['longitude'].values
    week_lat = this_df['latitude'].values
    week_data = [week_lon, week_lat]

    # Randomly select one profile:
    this_df = this_df.sample(1, random_state=12)
    print("Profile selected:\n", this_df, "\n")

    # ## Fetch all float data from ftp

    argo_file = this_df['file'].values[0].strip()
    # dac = df['file'].values[0].split('/')[0]
    # wmo = df['wmo'].values[0]
    # cmd = "rsync -avzh --delete vdmzrs.ifremer.fr::argo/%s/%s dac/%s" % (dac, wmo, dac) # Load all floats data
    # cmd = "rsync -avzh --delete vdmzrs.ifremer.fr::argo/%s dac/%s" % (df['file'].values[0].strip(), df['file'].values[0].strip()) # load only this profile
    with FTP("ftp.ifremer.fr") as ftp:
        ftp.login('anonymous','anonymous')
        cwd = '/ifremer/argo/dac/%s' % os.path.dirname(argo_file)
        ftp.cwd(cwd)
        with open(argo_file.split('/')[-1], "wb") as gFile:
            ftp.retrbinary('RETR %s' % argo_file.split('/')[-1], gFile.write)

    Path(os.path.dirname(os.path.sep.join(['dac', argo_file]))).mkdir(parents=True, exist_ok=True)
    # os.makedirs()
    move(argo_file.split('/')[-1], os.path.sep.join(['dac', argo_file]))

    return this_df, week_data

def open_this_profile(this_df):
    last_cycle = int(this_df['file'].values[0].split('/')[-1].split("_")[-1].split('.')[0])

    # ## Load data with argopy
    with argopy.set_options(local_ftp='.'):
        #     ds = ArgoDataFetcher(src='localftp', mode='expert').float([df['wmo'].values[0]]).to_xarray()
        this_ds = ArgoDataFetcher(src='localftp', mode='expert').profile([this_df['wmo'].values[0]], last_cycle).to_xarray()
        this_ds = this_ds.argo.point2profile()

    # Select the profile to work with:
    this_ds = this_ds.isel(N_PROF=-1)  # The last profile to date

    this_ds.attrs['quadrant'] = 'E' if this_ds['LONGITUDE'] > 0 else 'W'
    this_ds.attrs['hemisphere'] = 'N' if this_ds['LATITUDE'] > 0 else 'S'
    this_ds.attrs['float_model'] = mapp_dict(load_dict('profilers'), this_ds['WMO_INST_TYPE'].values[np.newaxis][0])

    return this_ds

def plot_profile(this_ds, x='TEMP', y='PRES', title='?', dpi=120):
    aspect, size = 0.6, 10
    # px, py = aspect * size * dpi, size * dpi pixels
    # print("width=", aspect * size * dpi, "height=", size * dpi, "pixels")
    fig, axes = plt.subplots(nrows=1, ncols=1, dpi=dpi, figsize=(aspect*size, size))
    this_ds.plot.scatter(ax=axes, x=x, y=y, s=2)
    ymin, ymax = axes.get_ylim()
    axes.set_ylim([0, ymax])
    axes.invert_yaxis()
    axes.set_title(title)

def plot_ts(this_ds, x='PSAL', y='TEMP', title='?', dpi=120):
    aspect, size = 1, 10
    # px, py = aspect * size * dpi, size * dpi pixels
    # print("width=", aspect * size * dpi, "height=", size * dpi, "pixels")
    fig, axes = plt.subplots(nrows=1, ncols=1, dpi=dpi, figsize=(aspect*size, size))
    this_ds.plot.scatter(ax=axes, x=x, y=y, s=2)
    # ymin, ymax = axes.get_ylim()
    # axes.set_ylim([0, ymax])
    # axes.invert_yaxis()
    axes.set_title(title)

def plot_map(this_ds, this_week_data, dpi=120, title='?'):
    week_lon, week_lat = this_week_data

    aspect, size = 1, 10
    fig = plt.figure(figsize=(aspect*size, size), dpi=dpi)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.add_feature(land_feature, edgecolor='black')

    # sns.lineplot(x="LONGITUDE", y="LATITUDE", hue="TIME", data=ds, sort=False, legend=False)
    # sns.scatterplot(x="LONGITUDE", y="LATITUDE", hue="TIME", data=ds, legend=False)

    plt.plot(week_lon, week_lat, 'k.', markersize=6)
    plt.plot(this_ds['LONGITUDE'], this_ds['LATITUDE'], 'r.', markersize=12)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.7, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    # width = np.abs(ds['LONGITUDE'].max() - ds['LONGITUDE'].min())
    # height = np.abs(ds['LATITUDE'].max() - ds['LATITUDE'].min())
    # width = 60
    # height = 60
    # extent = (ds['LONGITUDE'].min() - width / 4,
    #           ds['LONGITUDE'].max() + width / 4,
    #           ds['LATITUDE'].min() - height / 4,
    #           ds['LATITUDE'].max() + height / 4)
    extent = [-179.9, 180, -80, 80]
    ax.set_extent(extent)
    ax.set_title(title)


def add_lowerband(mfname, outfname, band_height=70, color=(255, 255, 255, 255)):
    """ Add lower band to a figure
        Parameters
        ----------
        mfname : string
            source figure file
        outfname : string
            output figure file
    """
    image = Image.open(mfname, 'r')
    image_size = image.size
    width = image_size[0]
    height = image_size[1]
    background = Image.new('RGBA', (width, height + band_height), color)
    background.paste(image, (0, 0))
    background.save(outfname)

def add_2logo(mfname, outfname, logo_height=70, font_size=12, txt_color=(0, 0, 0, 255)):
    """ Add 2 logos and text to a figure
        Parameters
        ----------
        mfname : string
            source figure file
        outfname : string
            output figure file
    """

    font_path = "img/Calibri_Regular.ttf"
    lfname2 = "img/logo_argofrance.png"
    lfname1 = "img/logo_argo.png"

    mimage = Image.open(mfname)

    # Open logo images:
    limage1 = Image.open(lfname1)
    limage2 = Image.open(lfname2)

    # Resize logos to match the requested logo_height:
    aspect_ratio = limage1.size[1] / limage1.size[0]  # height/width
    simage1 = limage1.resize((int(logo_height / aspect_ratio), logo_height))

    aspect_ratio = limage2.size[1] / limage2.size[0]  # height/width
    simage2 = limage2.resize((int(logo_height / aspect_ratio), logo_height))

    # Paste logos along the lower white band of the main figure:
    box = (0, mimage.size[1] - logo_height)
    mimage.paste(simage1, box)

    box = (simage1.size[0], mimage.size[1] - logo_height)
    mimage.paste(simage2, box)

    txtA = "Argo is an international program that collects information from inside " \
           "the ocean using a fleet of robotic instruments that drift with the ocean " \
           "currents and move up and down between the surface and a mid-water level.\n" \
           "These data were collected and made freely available by the International " \
           "Argo Program and the national programs that contribute to it. " \
           "(https://argo.ucsd.edu, https://www.ocean-ops.org). The Argo Program is " \
           "part of the Global Ocean Observing System.\n" \
           "This figure was created by the Argo-France program: https://www.argo-france.fr"
    xoffset = 5 + simage1.size[0] + simage2.size[0]

    def wrap_text(caption, max_width_pixel=200, font_size=12, font_path="?"):

        def with_ftsz(txt, ftsz=10, max_char=60):
            this_ft = ImageFont.truetype(font=font_path, size=ftsz)  # size – The requested size, in points.
            txt = textwrap.fill(txt, width=max_char, break_long_words=True, replace_whitespace=True)
            txt_sz = this_ft.getsize_multiline(txt)
            return txt, txt_sz, this_ft

        maxc = 50
        wrapped_txt, txt_sz, ft = with_ftsz(caption, ftsz=font_size, max_char=maxc)
        too_large = txt_sz[0] > max_width_pixel
        #     print(max_width_pixel, too_large, txt_sz[0], maxc)
        while not too_large:
            maxc += 1
            wrapped_txt, txt_sz, ft = with_ftsz(caption, ftsz=font_size, max_char=maxc)
            too_large = txt_sz[0] > max_width_pixel
        #         print(max_width_pixel, too_large, txt_sz[0], maxc)
        return wrapped_txt, ft

    txtA, fontA = wrap_text(txtA, font_size=font_size, max_width_pixel=mimage.size[0]-xoffset-50, font_path=font_path)

    # fontA = ImageFont.truetype(font=font_path, size=10) # size – The requested size, in points.
    # max_width = (image width - xoffset)/font_size_in_pixel
    # print(mimage.size[0], simage1.size[0], simage2.size[0])
    # max_char = (mimage.size[0] - xoffset )/13
    # print(max_char)
    # txtA = textwrap.fill(txtA, width=max_char)
    txtsA = fontA.getsize_multiline(txtA)
    # print(txtsA)

    if 1:  # Align text to the top of the band:
        posA = (xoffset, mimage.size[1] - logo_height - 1)
    else:  # Align text to the bottom of the band:
        posA = (xoffset, mimage.size[1] - txtsA[1] - 5)

    # Print
    drawA = ImageDraw.Draw(mimage)
    drawA.text(posA, txtA, txt_color, font=fontA)

    # Final save
    mimage.save(outfname)


def tweet_this(imlist, this_ds):
    
    # Init the Tweeter API
    consumer_key, consumer_secret, access_key, access_secret = os.environ['TWITTER_SECRET'].split(':')

    api = twitter.Api(consumer_key=consumer_key,
                      consumer_secret=consumer_secret,
                      access_token_key=access_key,
                      access_token_secret=access_secret)

    # ## Text of the tweet
    when = pd.to_datetime(str(this_ds['TIME'].values)).strftime('%A %d of %B %Y, %H:%M:%S UTC')
    where = "%0.3f%s, %0.3f%s" % (np.abs(this_ds['LATITUDE']), this_ds.attrs['hemisphere'], np.abs(this_ds['LONGITUDE']), this_ds.attrs['quadrant'])
    long_title = "▶ Float %s, Cycle %i 🗓%s 🗺%s 📏%i measurements 🤖%s 🔬%s 💳%s" % (
        wmo, ds['CYCLE_NUMBER'],
        when,
        where,
        len(this_ds['N_LEVELS']),
        this_ds.attrs['float_model'],
        this_ds['PI_NAME'].values[np.newaxis][0].strip().title(),
        this_ds['PROJECT_NAME'].values[np.newaxis][0].strip())

    # Could try to add a link to convert UTC to local, eg:
    # https://www.google.com/search?q=05%3A21%3A00+UTC+to+local
    # http://www.timebie.com/std/gmt.php?q=052100

    caption = (f'🏆 Here is the Argo profile of the last 24 hours 🎉\n'
               f'{long_title}\n'
               f'https://fleetmonitoring.euro-argo.eu/float/{wmo} '
               f'#argofloat')

    if len(caption) > twitter.api.CHARACTER_LIMIT:
        caption = caption[0:275] + '\u2026'

    print(len(caption), "\n", caption, "\n")

    # ## Attach figures to the tweet
    media = []
    for fname in imlist:
        with open(fname, 'rb') as f:
            media.append(api.UploadMediaChunked(f))

    # Post it !
    # api.PostUpdate(caption, media=media)
    resp = api.PostUpdate(caption,
                          media=media,
                          latitude=float(this_ds['LATITUDE'].values), longitude=float(this_ds['LONGITUDE'].values), display_coordinates=True)
    return resp

In [3]:
# DATE_end = pd.to_datetime('now', utc=True) # Throw: TypeError: Cannot compare tz-naive and tz-aware timestamps at argo_index.py / line 402
DATE_end = pd.to_datetime('now', utc=False)
DATE_start = DATE_end - pd.DateOffset(hours=24.0)
# print(DATE_start)
df, week_data = download_one_profile_data(DATE_start, DATE_end)
ds = open_this_profile(df)

# # Create and save figures
dac = df['file'].values[0].split('/')[0]
wmo = df['wmo'].values[0]
# when = pd.to_datetime(str(this_ds['TIME'].values)).strftime('%Y.%m.%dT%H:%M:%S')
when = pd.to_datetime(str(ds['TIME'].values)).strftime('%A %d of %B %Y, %H:%M:%S UTC')
where = "%0.3f%s, %0.3f%s" % (np.abs(ds['LATITUDE']), ds.attrs['hemisphere'], np.abs(ds['LONGITUDE']), ds.attrs['quadrant'])
profile_title = "Float %s, cycle %i\nDate: %s\nPosition: %s" % (wmo, ds['CYCLE_NUMBER'], when, where)
# print(profile_title, "\n")

# ## Plots
vnames = ['TEMP', 'PSAL', 'CNDC', 'DOXY', 'BBP', 'TURBIDITY', 'CHLA', 'CDOM', 'NITRATE', 'BISULFIDE', 'PH_IN_SITU_TOTAL', 'DOWNWELLING_PAR']
imlist = []
for v in vnames:
    if v in ds:
        plot_profile(ds, x=v, y='PRES', title=profile_title, dpi=240)
        iname = '%s.png' % v
        plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
        add_lowerband(iname, iname, band_height=100)
        add_2logo(iname, iname, logo_height=100, font_size=18)
        imlist.append(iname)

if 'TEMP' in ds and 'PSAL' in ds:
    plot_ts(ds, x='PSAL', y='TEMP', title=profile_title, dpi=240)
    iname = 'TS.png'
    plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
    add_lowerband(iname, iname, band_height=100)
    add_2logo(iname, iname, logo_height=100, font_size=18)
    imlist.append(iname)

plot_map(ds, week_data, dpi=120, title=profile_title)
iname = 'MAP.png'
plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
add_lowerband(iname, iname, band_height=100)
add_2logo(iname, iname, logo_height=100, font_size=12)
imlist.append(iname)

# Tweet !
tweet_this(imlist, ds)

329 Argo profiles reported between 2021-07-05 14:51:03.334251 and 2021-07-06 14:51:03.334251
Profile selected:
                                       file                date   latitude  \
190  bodc/6901192/profiles/R6901192_183.nc 2021-07-06 00:14:50  53.206001   

     longitude ocean profiler_type institution         date_update      wmo  
190     -37.91     A           846          BO 2021-07-06 08:07:16  6901192   

264 
 🏆 Here is the Argo profile of the last 24 hours 🎉
▶ Float 6901192, Cycle 183 🗓Tuesday 06 of July 2021, 00:14:50 UTC 🗺53.206N, 37.910W 📏57 measurements 🤖Webb Research, Seabird sensor 🔬Jon Turton 💳Argo UK
https://fleetmonitoring.euro-argo.eu/float/6901192 #argofloat 



Status(ID=1412423905650876432, ScreenName=argobot84, Created=Tue Jul 06 14:51:15 +0000 2021, Text='🏆 Here is the Argo profile of the last 24 hours 🎉\n▶ Float 6901192, Cycle 183 🗓Tuesday 06 of July 2021, 00:14:50 UTC… https://t.co/fHGCWhynbl')

***
This repository is maintained by:
<div>
<img src="https://www.argo-france.fr/wp-content/uploads/2019/10/Argo-logo_banner-color.png" width="200"/>
<img src="https://www.umr-lops.fr/var/storage/images/_aliases/logo_main/medias-ifremer/medias-lops/logos/logo-lops-2/1459683-4-fre-FR/Logo-LOPS-2.png" width="70"/>
</div>